In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import pandas as pd
from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
# from sklearn.preprocessing import normalize


In [2]:
# df_selected = df[['模块', '严重度', '问题现象描述', '原因分析', '改善对策', '经验萃取', '评审后优化', '评分']]
# df_selected.columns = ['module', 'severity', 'question', 'cause', 'improvement', 'experience', 'judge', 'score']
# loader = CSVLoader(file_path="../../../doc/dsq_learn_sheet_eng_titles.csv")
# data = loader.load()
# data_len = len(data)
# print(data_len)


In [3]:
# print(data[0])

In [4]:
# def embedding_DQE(csv_file=None,qKey="Question", qAns="Answer"):
#     df = pd.read_csv(csv_file)
#     # 合併重複問題的答案
#     df_clean = df.groupby(qKey)[qAns].apply(lambda x: '\n'.join(x.unique())).reset_index()
#     # 正規化代碼格式
#     """
#     這是一個使用正則表達式的字串替換操作。目的是將Answer欄位中的參數設定格式標準化。
#     例如，將類似"PcdCfgPeApmEnable = 0"中的空格去除，變成"PcdCfgPeApmEnable=0"。
#     這樣做的好處是統一格式，避免後續處理時因格式不一致導致問題，比如在檢索或解析時無法正確識別參數和數值。
#     """
#     df_clean[qAns] = df_clean[qAns].str.replace(r'(\w+)\s*=\s*(\d+)', r'\1=\2', regex=True)
#     # 提取文本數據
#     questions = df_clean[qKey].str.strip().tolist()
#     answers = df_clean[qAns].str.strip().tolist()
#     model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
#     question_embeddings = model.encode(
#         questions,
#         convert_to_tensor=False,
#         show_progress_bar=True
#     )
#     print(question_embeddings.shape)
#     dimension = question_embeddings.shape[1]#(#__1)  # 向量維度(通常為384/768)
#     quantizer = faiss.IndexFlatL2(dimension)
#     index = faiss.IndexIVFFlat(quantizer, dimension, 100)
#     index.train(question_embeddings)
#     index.add(question_embeddings)
#     faiss.write_index(index, "qa_index_.faiss")
#     #integrated with langchain
#     # 創建元數據結構
#     metadatas = [{
#         "answer": ans, 
#         "source": "內部技術資料庫",
#         "last_updated": "2025-02"
#     } for ans in answers]
#     # 建立可持久化向量庫
#     vector_db = FAISS.from_embeddings(
#         text_embeddings=list(zip(questions, question_embeddings)),
#         embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"),
#         metadatas=metadatas
#     )
#     # 保存完整向量庫
#     vector_db.save_local("tech_support_faiss")

### Start to Embed the DEQ Learn 28 data

In [5]:
# df_selected = df[['模块', '严重度', '问题现象描述', '原因分析', '改善对策', '经验萃取', '评审后优化', '评分']]
# df_selected.columns = ['module', 'severity', 'question', 'cause', 'improvement', 'experience', 'judge', 'score']
# df = pd.read_csv('../../../doc/deq_learn_refine2_correct.csv');
# print(len(df))

In [6]:
key_question_src = "questionsource"
key_question = "question"
key_improve = "improve"
key_severity = "severity"
key_module = "module"
key_cause = "cause"
key_experience="experience"
key_judge = "judge"
key_score = "score"
key_shortquestion = "shortquestion"
key_shortanswer = "shortanswer"
title_mapping = {
    key_question_src:'问题来源',
    key_module:'模块',
    key_severity:'严重度',
    key_question:'问题现象描述',
    key_cause:'原因分析',
    key_improve:'改善对策',
    key_experience:'经验萃取',
    key_judge:'评审后优化',
    key_score:'评分',
}
deq_df = pd.read_csv("../../../doc/DQE_Lesson_learn_AI_0311_five_refine_question - AI_Trial Run.csv");
questions_src = deq_df[key_question_src].str.strip().tolist();
questions = deq_df[key_question].str.strip().tolist();
severity = deq_df[key_severity].str.strip().tolist();
modules = deq_df[key_module].str.strip().tolist();
causes = deq_df[key_cause].str.strip().tolist();
improves = deq_df[key_improve].str.strip().tolist();
experiences = deq_df[key_experience].str.strip().tolist();
judges = deq_df[key_judge].str.strip().tolist();
scores = deq_df[key_score].tolist(); #this is int
shortquestions = deq_df[key_shortquestion].str.strip().tolist();
shortanswers= deq_df[key_shortanswer].str.strip().tolist();

In [7]:
def embed_index_for_multicolumns(encode_model:str="paraphrase-multilingual-MiniLM-L12-v2"):
    
    if len(questions)!= len(scores) or len(questions)!= len(severity) \
        or len(questions)!= len(modules) or len(questions)!= len(causes):
        raise ValueError("To embedded data is not equal.")
    model = SentenceTransformer(encode_model);
    # embedding two columns: questions_src and  module
    question_src_embeddings = model.encode(
        questions_src,
        convert_to_tensor=False,
        show_progress_bar=True
    )
    module_embeddings = model.encode(
        modules,
        convert_to_tensor=False,
        show_progress_bar=True
    )
    print(f"the encoded question shape is {question_src_embeddings.shape}");
    combined_embeddings = np.concatenate([question_src_embeddings, module_embeddings], axis=1)
    dimension = combined_embeddings.shape[1]
    quantizer = faiss.IndexFlatL2(dimension)
    # some the same issues:
    # https://github.com/facebookresearch/faiss/issues/1637
    print(f"len of combined_embeddings is {len(combined_embeddings)}")
    index = faiss.IndexIVFFlat(quantizer, dimension, len(combined_embeddings))
    index.train(combined_embeddings)
    index.add(combined_embeddings)
    faiss.write_index(index, "../../../web/db/20250317/faiss_index_qsrc_mo.faiss")
    metadatas = [];
    for idx in range(len(combined_embeddings)):
        tmpdict = {
                "question": questions[idx],
                "severity": severity[idx],
                "module": modules[idx],
                "cause":causes[idx],
                "improve":improves[idx],
                "experience":experiences[idx],
                "judge":judges[idx],
                "score":scores[idx],
        }
        metadatas.append(tmpdict)

    # print(metadatas);
    # 建立可持久化向量庫
    combined_cols = np.concatenate([questions_src, modules], axis=0)
    print(f"combined_cols are:{combined_cols}")
    vector_db = FAISS.from_embeddings(
        text_embeddings=list(zip(combined_cols, combined_embeddings)),
        embedding=HuggingFaceEmbeddings(model_name=encode_model),
        metadatas=metadatas
    )
    # 保存完整向量庫
    vector_db.save_local("../../../web/db/20250317/faiss_vdb_qsrc_mo")

In [27]:
embed_index_for_multicolumns()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

the encoded question shape is (5, 384)
len of combined_embeddings is 5
combined_cols are:['EVT' 'PVT' '首次量产' '量产' 'EVT' '结构' 'ESD' '工艺' '工艺' '软件']


WARNING clustering 5 points to 5 centroids: please provide at least 195 training points


In [17]:
def embed_index_for_singlecolumns(encode_model:str="paraphrase-multilingual-MiniLM-L12-v2"):
    
    if len(questions)!= len(scores) or len(questions)!= len(severity) \
        or len(questions)!= len(modules) or len(questions)!= len(causes):
        raise ValueError("To embedded data is not equal.")
    model = SentenceTransformer(encode_model);
    # embedding two columns: questions_src and  module
    module_embeddings = model.encode(
        modules,
        convert_to_tensor=False,
        show_progress_bar=True
    )
    print(f"the encoded modules shape is {module_embeddings.shape}");
    
    dimension = module_embeddings.shape[1]
    quantizer = faiss.IndexFlatL2(dimension)
    # some the same issues:
    # https://github.com/facebookresearch/faiss/issues/1637
    print(f"len of module_embeddings is {len(module_embeddings)}")
    index = faiss.IndexIVFFlat(quantizer, dimension, len(module_embeddings))
    index.train(module_embeddings)
    index.add(module_embeddings)
    faiss.write_index(index, "../../../web/db/20250317/faiss_index_module.faiss")

In [18]:
embed_index_for_singlecolumns()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

the encoded modules shape is (5, 384)
len of module_embeddings is 5


WARNING clustering 5 points to 5 centroids: please provide at least 195 training points


In [ ]:
def embed_DEQ_Learn_from_csv(csv_file:str=None, paras:dict=None, encode_model:str="paraphrase-multilingual-MiniLM-L12-v2"):
    df = pd.read_csv(csv_file);
    # df_clean = df.groupby(question_key)[improve_key].apply(lambda x: '\n'.join(x.unique())).reset_index()
    questions = df[key_question].str.strip().tolist();
    severity = df[key_severity].str.strip().tolist();
    modules = df[key_module].str.strip().tolist();
    causes = df[key_cause].str.strip().tolist();
    improves = df[key_improve].str.strip().tolist();
    experiences = df[key_experience].str.strip().tolist();
    judges = df[key_judge].str.strip().tolist();
    scores = df[key_score].tolist(); #this is int
    if len(questions)!= len(scores) or len(questions)!= len(severity) \
        or len(questions)!= len(modules) or len(questions)!= len(causes):
        raise ValueError("To embedded data is not equal.")
    model = SentenceTransformer(encode_model);
    # embedding question
    question_embeddings = model.encode(
        questions,
        convert_to_tensor=False,
        show_progress_bar=True
    )
    print(f"the encoded question shape is {question_embeddings.shape}");
    dimension = question_embeddings.shape[1]
    quantizer = faiss.IndexFlatL2(dimension)
    # some the same issues:
    # https://github.com/facebookresearch/faiss/issues/1637
    index = faiss.IndexIVFFlat(quantizer, dimension, 28)
    index.train(question_embeddings)
    index.add(question_embeddings)
    faiss.write_index(index, "../../../db/20250307v1/dqe_learn_vdb_index.faiss")
    #integrated with langchain
    # 創建元數據結構
    metadatas = [];
    for idx in range(len(questions)):
        tmpdict = {
                "severity": severity[idx],
                "module": modules[idx],
                "cause":causes[idx],
                "improve":improves[idx],
                "experience":experiences[idx],
                "judge":judges[idx],
                "score":scores[idx],
        }
        metadatas.append(tmpdict)

    # print(metadatas);
    # 建立可持久化向量庫
    vector_db = FAISS.from_embeddings(
        text_embeddings=list(zip(questions, question_embeddings)),
        embedding=HuggingFaceEmbeddings(model_name=encode_model),
        metadatas=metadatas
    )
    # 保存完整向量庫
    vector_db.save_local("../../../db/20250307v1/dqe_learn_vdb_faiss")

In [62]:
_csv_file= "../../../doc/dsq_learn_sheet_eng_titles.csv"
embed_DEQ_Learn_from_csv(csv_file=_csv_file)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

the encoded question shape is (28, 384)


WARNING clustering 28 points to 28 centroids: please provide at least 1092 training points


In [53]:
def embedding_DQE(csv_file=None,qKey="Question", qAns="Answer"):
    df = pd.read_csv(csv_file)
    # 合併重複問題的答案
    df_clean = df.groupby(qKey)[qAns].apply(lambda x: '\n'.join(x.unique())).reset_index()
    # 正規化代碼格式
    """
    這是一個使用正則表達式的字串替換操作。目的是將Answer欄位中的參數設定格式標準化。
    例如，將類似"PcdCfgPeApmEnable = 0"中的空格去除，變成"PcdCfgPeApmEnable=0"。
    這樣做的好處是統一格式，避免後續處理時因格式不一致導致問題，比如在檢索或解析時無法正確識別參數和數值。
    """
    df_clean[qAns] = df_clean[qAns].str.replace(r'(\w+)\s*=\s*(\d+)', r'\1=\2', regex=True)
    # 提取文本數據
    questions = df_clean[qKey].str.strip().tolist()
    answers = df_clean[qAns].str.strip().tolist()
    model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
    question_embeddings = model.encode(
        questions,
        convert_to_tensor=False,
        show_progress_bar=True
    )
    print(question_embeddings.shape)
    dimension = question_embeddings.shape[1]#(#__1)  # 向量維度(通常為384/768)
    quantizer = faiss.IndexFlatL2(dimension)
    index = faiss.IndexIVFFlat(quantizer, dimension, 100)
    index.train(question_embeddings)
    index.add(question_embeddings)
    faiss.write_index(index, "qa_index_.faiss")
    #integrated with langchain
    # 創建元數據結構
    metadatas = [{
        "answer": ans, 
        "source": "內部技術資料庫",
        "last_updated": "2025-02"
    } for ans in answers]
    # 建立可持久化向量庫
    vector_db = FAISS.from_embeddings(
        text_embeddings=list(zip(questions, question_embeddings)),
        embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"),
        metadatas=metadatas
    )
    # 保存完整向量庫
    vector_db.save_local("tech_support_faiss")